In [10]:
import pandas as pd
import numpy as np
import collections as co
import statistics as st

from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.models import load_model

from bayes_opt import BayesianOptimization


import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from matplotlib import ticker
import matplotlib.pyplot as plt



In [9]:
#pip install bayesian-optimization

  Using cached bayesian_optimization-1.4.3-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv('./Data/CTLT.csv')
df = df.dropna() # Drop missing values
#df = df.reset_index(drop=True) # Reset the index

In [3]:
X = df[['Open', 'High', 'Low', 'Volume']].values
y = df['Close'].values

# X_train = np.array(train_data)

In [4]:
# chia tập dữ liệu ban đầu thành 2 phần là tập train và tập test với tỷ lệ 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# tập test được chia thành 2 phần là tập validation và tập test với tỷ lệ 
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=42)


train_size = len(X_train)
val_size = len(X_val)
test_size = len(X_test)

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

# Split the data into training, testing, and validation sets
# train_size = int(0.7 * len(df))
# test_size = int(0.2 * len(df))
# val_size = len(df) - train_size - test_size

# train_data = df[:train_size]
# test_data = df[train_size:train_size+test_size]
# val_data = df[train_size+test_size:]

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)



In [20]:
# define objective function
def objective_function(lr, num_hidden1, num_hidden2, decay):
    model = Sequential()
    model.add(Dense(num_hidden1, activation='relu'))
    model.add(Dense(num_hidden2, activation='relu'))
    model.add(Dense(1))
    optimizer = Adam(lr=lr, decay=decay)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    kf = KFold(n_splits=5)
    val_loss = []
    for train_idx, val_idx in kf.split(X_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, verbose=0)
        val_loss.append(model.evaluate(X_val_fold, y_val_fold, verbose=0))
    return np.mean(val_loss)

# define search space
pbounds = {'lr': (0.001, 0.1),
           'num_hidden1': (50, 200),
           'num_hidden2': (50, 200),
           'decay': (0.001, 0.1)}

# define optimizer
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=1)

# perform optimization
optimizer.maximize(n_iter=10)

# print best hyperparameters
print(optimizer.max)

|   iter    |  target   |   decay   |    lr     | num_hi... | num_hi... |
-------------------------------------------------------------------------


NameError: name 'Adam' is not defined